# Mini-Projet Intelligence Artificielle : Prédiction du Revenu Annuel d'un Marocain

**Membres du Groupe :**
*   Saad Barhrouj
*   Nassim El Kaddaoui
*   Youness Kihel

**Année :** 2ème année Cycle d’Ingénieurs – GI
**Module :** Intelligence Artificielle
**Encadrant :** Y. EL YOUNOUSSI
**Année Universitaire :** 2024-2025

## Objectif du Mini-Projet
L'objectif général de ce mini-projet est de construire un pipeline complet de Machine Learning en Python pour prédire le revenu annuel des Marocains à partir de données simulées réalistes. Le projet couvre toutes les étapes de développement d’un modèle de Machine Learning : compréhension des données, préparation des données, modélisation, évaluation et déploiement.

# 0. Génération du Dataset Synthétique (`dataset_revenu_marocains.csv`)

Avant de pouvoir analyser et modéliser, nous avons dû générer un dataset synthétique simulant les revenus des Marocains. Cette étape a été réalisée via un script Python séparé (`generate_dataset.py`). Ci-dessous, nous résumons la méthodologie et les principales logiques implémentées dans ce script.

**Objectif du script `generate_dataset.py` :**
Créer un fichier CSV (`dataset_revenu_marocains.csv`) contenant environ 40 000 enregistrements, en respectant des contraintes déduites des statistiques du HCP et des spécifications du projet.

**Principales caractéristiques du dataset généré :**
*   **Variable Cible :** `Revenu_Annuel`.
*   **Caractéristiques Explicatives :** Incluent `Age`, `Categorie_age`, `Milieu` (Urbain/Rural), `Sexe`, `Niveau_education`, `Annees_experience`, `Etat_matrimonial`, `Possession_biens` (Immobilier, Véhicule, Terrain), `CSP`, ainsi que des caractéristiques supplémentaires (`Region_geographique`, `Secteur_emploi`, `Revenu_secondaire`).
*   **Imperfections Intégrées :** Le dataset inclut des valeurs manquantes, des valeurs aberrantes, des colonnes redondantes et des colonnes non pertinentes, conformément aux exigences du projet.

## 0.1 Logique de Génération et Statistiques Clés (Résumé)

Le script `generate_dataset.py` utilise `RANDOM_SEED = 42` pour la reproductibilité et génère `N_RECORDS = 40000`.

**Logique de Génération (Principes Généraux) :**
1.  **Caractéristiques de base :** Des attributs comme `Age`, `Milieu`, `Sexe`, et `Niveau_education` sont générés en premier, souvent en utilisant des distributions de probabilités prédéfinies (ex: `P_URBAIN = 0.60`).
2.  **Caractéristiques dépendantes :** D'autres attributs tels que `Annees_experience` (basée sur l'âge et l'éducation), `Etat_matrimonial` (basé sur l'âge), `CSP` (basée sur l'éducation, l'expérience, l'âge), et la `Possession_biens` (basée sur la CSP et le milieu) sont générés de manière à refléter des interdépendances logiques.
3.  **Revenu Annuel (Cible) :** La génération du revenu est la plus complexe. Elle part d'un revenu de base (issu d'une distribution log-normale) et l'ajuste en fonction de multiples facteurs (Milieu, Éducation, Expérience, CSP, Sexe, Région, etc.) à travers des bonus, malus, ou facteurs multiplicatifs. Un plancher et un plafond sont appliqués.
4.  **Imperfections :**
    *   **Valeurs manquantes (`NaN`) :** Introduites aléatoirement (faible pourcentage) dans certaines colonnes (ex: `Etat_matrimonial`, `Secteur_emploi`, `Possession_biens`).
    *   **Valeurs aberrantes :** Ajoutées sur `Age` (ex: -5, 150), `Annees_experience` (ex: expérience > âge), `Possession_biens` (ex: inactif avec beaucoup de biens), et `Revenu_Annuel` (revenus extrêmes ou incohérents avec la CSP).
    *   **Colonnes redondantes/non pertinentes :** `Revenu_Mensuel` (calculé à partir de `Revenu_Annuel`), `Adresse_Email` et `CIN` (identifiants uniques).
5.  **Colonne dérivée :** `Categorie_age` est créée à partir de `Age`.

**Statistiques Clés du Dataset `dataset_revenu_marocains.csv` (issues de l'exécution de `generate_dataset.py`) :**
*   **Nombre d'enregistrements :** 40,000
*   **Revenu Annuel Moyen (global) :** 23,234.97 DH
*   **Écart-type du Revenu Annuel :** 31,689.44 DH
*   **Revenu Annuel (Min | Max) :** 200 DH | 700,000 DH
*   **Moyenne Revenu (Urbain | Rural) :** 23,963.65 DH | 22,149.10 DH
*   **Classement des revenus moyens par CSP (décroissant) :** Cadres supérieurs > Professions intermédiaires > Employés > Ouvriers > Inactifs > Agriculteurs.
*   **Classement des revenus moyens par Niveau d'éducation (décroissant) :** Supérieur > Secondaire > Fondamental > Sans niveau.

Pour une compréhension exhaustive de la logique de génération, le script `generate_dataset.py` reste la référence. Le dataset ainsi créé est celui que nous allons charger et analyser dans les sections suivantes.

# 1. Compréhension des Données

Après avoir résumé le processus de génération du dataset, nous allons maintenant charger le fichier `dataset_revenu_marocains.csv` et l'explorer en détail. L'objectif est de bien comprendre sa structure, son contenu, et d'identifier les caractéristiques notables avant de passer à la préparation des données pour la modélisation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pour la reproductibilité des opérations dans CE notebook (si nécessaire plus tard)
RANDOM_SEED = 42 # La même que dans votre script de génération
np.random.seed(RANDOM_SEED)
# import random # Si vous utilisez le module random de Python
# random.seed(RANDOM_SEED)

# Options d'affichage (optionnel, pour une meilleure lisibilité)
pd.set_option('display.max_columns', None) # Afficher toutes les colonnes
pd.set_option('display.max_rows', 100)    # Afficher jusqu'à 100 lignes
pd.set_option('display.width', 1000)      # Ajuster la largeur d'affichage
# pd.set_option('display.float_format', lambda x: '%.2f' % x) # Formater les flottants si besoin

# Pour des graphiques plus esthétiques (optionnel)
# Vous pouvez choisir un style qui vous plaît
# plt.style.use('ggplot')
# plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-whitegrid') # Celui-ci est souvent bien
sns.set_palette("pastel") # Ou "viridis", "muted", "colorblind", etc.

# Ignorer les avertissements pour un notebook plus propre (optionnel)
import warnings
warnings.filterwarnings('ignore')

print("Librairies pour l'analyse de données importées.")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Seaborn version: {sns.__version__}")
# print(f"Matplotlib version: {matplotlib.__version__}") # Nécessite 'import matplotlib'

Librairies pour l'analyse de données importées.
Pandas version: 2.2.3
Numpy version: 2.1.3
Seaborn version: 0.13.2


## 1.1 Chargement du Dataset
Nous chargeons le fichier `dataset_revenu_marocains.csv` qui a été généré par notre script `generate_dataset.py`. Ce fichier contient les données simulées sur lesquelles nous allons travailler.

In [2]:
# Spécifiez le nom du fichier CSV généré.
# Assurez-vous que ce fichier est dans le même répertoire que votre notebook,
# ou fournissez le chemin complet vers le fichier.
FILENAME = "dataset_revenu_marocains.csv"
df = None # Initialiser pour une gestion propre des erreurs potentielles

try:
    df = pd.read_csv(FILENAME)
    print(f"Le fichier '{FILENAME}' a été chargé avec succès dans le DataFrame 'df'.")
    print(f"Dimensions du DataFrame : {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"ERREUR CRITIQUE : Le fichier '{FILENAME}' n'a pas été trouvé.")
    print("Veuillez vérifier que le script de génération a bien été exécuté et que le fichier CSV est au bon emplacement (généralement le même dossier que ce notebook).")
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")
except Exception as e:
    print(f"Une erreur est survenue lors du chargement du fichier '{FILENAME}' :")
    print(e)
    print("Les étapes suivantes ne pourront pas être exécutées sans le chargement des données.")

# Vérification rapide pour confirmer que le DataFrame est chargé avant de continuer
if df is not None:
    print("\nLe DataFrame 'df' est prêt pour l'analyse.")
else:
    print("\nATTENTION : Le DataFrame 'df' n'a pas été chargé. Veuillez corriger le problème avant de continuer.")

Le fichier 'dataset_revenu_marocains.csv' a été chargé avec succès dans le DataFrame 'df'.
Dimensions du DataFrame : 40000 lignes et 18 colonnes.

Le DataFrame 'df' est prêt pour l'analyse.


## 1.2 Affichage des Données (Aperçu)
Pour avoir une première idée concrète du contenu et de la structure des données, affichons les 10 premières et les 10 dernières lignes du DataFrame. Cela nous permet de vérifier rapidement les noms des colonnes et le format des données.

In [3]:
if df is not None:
    print("Aperçu des 10 premières lignes du DataFrame :")
    # display() est souvent préférable à print() pour les DataFrames dans Jupyter car il offre un meilleur rendu HTML.
    display(df.head(10))

    print("\nAperçu des 10 dernières lignes du DataFrame :")
    display(df.tail(10))
else:
    # Ce message ne devrait pas s'afficher si le chargement précédent a réussi.
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les données.")

Aperçu des 10 premières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
0,56,Senior,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non,5231.0,435.92,user8ded3b@example.com,YF632342
1,46,Senior,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non,86759.0,7229.92,userbfb22c@example.com,DU412942
2,32,Adulte,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non,3941.0,328.42,user2414ad@example.com,UQ738551
3,60,Âgé,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,109176.0,9098.00,user5758fb@example.com,GE492077
4,25,Adulte,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non,14906.0,1242.17,user9daf81@example.com,YF665579
5,38,Adulte,Homme,Rural,Est,Divorcé,Secondaire,5.0,Employés,Privé,Oui,Non,Non,Non,15206.0,1267.17,userb79944@example.com,YQ100599
6,56,Senior,Homme,Rural,Ouest,Marié,Fondamental,33.0,Ouvriers,Privé,Non,Non,Non,Non,9408.0,784.00,user3cf929@example.com,TK612340
7,36,Adulte,Femme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Oui,Oui,Non,Non,17057.0,1421.42,userec9691@example.com,AD480612
8,40,Adulte,Homme,Urbain,Centre,Marié,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Non,Non,Non,109897.0,9158.08,userd88bda@example.com,ZJ351083
9,28,Adulte,Femme,Urbain,Nord,Marié,Secondaire,3.0,Ouvriers,Privé,Non,Non,Non,Non,10560.0,880.00,useraa4212@example.com,BH694916



Aperçu des 10 dernières lignes du DataFrame :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
39990,26,Adulte,Homme,Urbain,Nord,Veuf,Secondaire,5.0,Employés,Public,Non,Non,Non,Oui,21343.0,1778.58,user81d35c@example.com,YJ411569
39991,40,Adulte,Homme,Rural,Nord,Célibataire,Secondaire,1.0,Ouvriers,Privé,Non,Non,Non,Non,10634.0,886.17,userf77e5a@example.com,PN139847
39992,24,Jeune,Femme,Urbain,Sud,Célibataire,Fondamental,1.0,Agriculteurs,Informel,Non,Non,Oui,Oui,6178.0,514.83,userf4e988@example.com,UP616808
39993,48,Senior,Femme,Urbain,Centre,Marié,Secondaire,25.0,Professions intermédiaires,Public,Non,Non,Oui,Non,35204.0,2933.67,user0b9a23@example.com,TQ978793
39994,38,Adulte,Femme,Urbain,Nord,Célibataire,Supérieur,13.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui,86262.0,7188.50,userf7ace0@example.com,SQ690182
39995,18,Jeune,Femme,Urbain,Centre,Marié,Sans niveau,0.0,Inactifs,NaN,Non,Non,Non,Non,4043.0,336.92,usera1411f@example.com,MV264884
39996,63,Âgé,Homme,Urbain,Ouest,Divorcé,Supérieur,30.0,Cadres supérieurs,Public,Non,Oui,Non,Oui,178995.0,14916.25,user45eb54@example.com,QN622528
39997,19,Jeune,Femme,Rural,Sud,Célibataire,Fondamental,0.0,Inactifs,NaN,Non,Non,Non,Non,3647.0,303.92,user840ba4@example.com,JG342827
39998,57,Senior,Homme,Urbain,Nord,Marié,Secondaire,21.0,Professions intermédiaires,Privé,Non,Non,Non,Oui,43689.0,3640.75,user53f996@example.com,GW748819
39999,28,Adulte,Homme,Urbain,Nord,Marié,Sans niveau,1.0,Inactifs,NaN,Oui,Non,Non,Oui,7993.0,666.08,user8982fc@example.com,UR878598


## 1.3 Description Générale des Données (Types et Valeurs Manquantes)
Pour obtenir une vue synthétique des types de données de chaque colonne et du nombre de valeurs non nulles (ce qui nous indique les valeurs manquantes), nous utilisons la méthode `info()` du DataFrame.

In [4]:
if df is not None:
    print("Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :")
    df.info()
else:
    # Ce message ne devrait pas s'afficher si le chargement et l'affichage précédents ont réussi.
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible d'afficher les informations.")

Informations générales sur le DataFrame (types, nombre de valeurs non nulles par colonne) :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    40000 non-null  int64  
 1   Categorie_age          39960 non-null  object 
 2   Sexe                   40000 non-null  object 
 3   Milieu                 40000 non-null  object 
 4   Region_geographique    40000 non-null  object 
 5   Etat_matrimonial       39965 non-null  object 
 6   Niveau_education       40000 non-null  object 
 7   Annees_experience      39960 non-null  float64
 8   CSP                    40000 non-null  object 
 9   Secteur_emploi         37390 non-null  object 
 10  Propriete_immobiliere  39958 non-null  object 
 11  Vehicule_motorise      39965 non-null  object 
 12  Terrain_agricole       39969 non-null  object 
 13  Revenu_seconda

## 1.4 Statistiques Descriptives
Nous allons maintenant calculer les statistiques descriptives (moyenne, écart-type, min, max, quartiles, etc.) pour les variables numériques, et les statistiques de fréquence (nombre d'uniques, valeur la plus fréquente) pour les variables catégorielles.

In [5]:
if df is not None:
    print("Statistiques descriptives pour les colonnes numériques :")
    # Sélectionner explicitement les colonnes numériques pour la méthode describe()
    # L'utilisation de .transpose() ou .T améliore la lisibilité lorsque vous avez de nombreuses colonnes numériques.
    numeric_stats = df.select_dtypes(include=np.number).describe().T
    display(numeric_stats)

    print("\nStatistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :")
    # Inclure 'category' au cas où 'Categorie_age' serait de ce type (ou si vous convertissez d'autres colonnes plus tard)
    categorical_stats = df.select_dtypes(include=['object', 'category']).describe().T
    display(categorical_stats)

    print("\nNombre de valeurs uniques par colonne (trié par ordre décroissant) :")
    # .to_frame() est utilisé pour un meilleur affichage d'une Series
    unique_counts = df.nunique().sort_values(ascending=False).to_frame(name='Nombre de Valeurs Uniques')
    display(unique_counts)
else:
    # Ce message ne devrait pas s'afficher.
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé. Impossible de calculer les statistiques.")

Statistiques descriptives pour les colonnes numériques :


,count,mean,std,min,25%,50%,75%,max
Age,40000.0,40.512625,13.509623,-5.00,29.00,40.00,52.00,150.00
Annees_experience,39960.0,10.401051,9.890290,0.00,2.00,8.00,16.00,45.00
Revenu_Annuel,40000.0,23234.973550,31689.439974,200.00,5155.00,9419.50,30203.00,700000.00
Revenu_Mensuel,40000.0,1936.247798,2640.786657,16.67,429.58,784.96,2516.92,58333.33



Statistiques descriptives pour les colonnes catégorielles (type 'object' et 'category') :


,count,unique,top,freq
Categorie_age,39960,4,Adulte,17615
Sexe,40000,2,Homme,20817
Milieu,40000,2,Urbain,23937
Region_geographique,40000,5,Centre,11212
Etat_matrimonial,39965,4,Marié,21297
Niveau_education,40000,4,Secondaire,13974
CSP,40000,6,Ouvriers,14975
Secteur_emploi,37390,3,Privé,17624
Propriete_immobiliere,39958,2,Non,29202
Vehicule_motorise,39965,2,Non,28477



Nombre de valeurs uniques par colonne (trié par ordre décroissant) :


,Nombre de Valeurs Uniques
CIN,39999
Adresse_Email,39964
Revenu_Mensuel,23683
Revenu_Annuel,23683
Age,48
Annees_experience,46
CSP,6
Region_geographique,5
Categorie_age,4
Etat_matrimonial,4


## 1.5 Exploration Visuelle Approfondie (EDA avec ydata-profiling)
Pour obtenir une compréhension encore plus profonde et visuelle des données, notamment les distributions de chaque variable, les corrélations, les interactions et les valeurs manquantes, nous allons utiliser la librairie `ydata-profiling` (anciennement Pandas Profiling). Elle génère un rapport HTML interactif qui fournit une analyse détaillée de chaque variable du dataset.


In [9]:
if 'df' in locals() and df is not None: # Vérifier que df existe et est chargé
    try:
        from ydata_profiling import ProfileReport
        
        print("Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.")
        
        # Créer le rapport de profilage SANS dark_mode et orange_mode directement ici
        profile = ProfileReport(df, 
                                title="Rapport d'Exploration Détaillée - Revenu des Marocains",
                                explorative=True) # explorative=True est toujours une bonne option
        
        # Sauvegarder le rapport en fichier HTML
        profile_filename = "Pandas_Profiling_Report_Revenu_Marocains.html"
        profile.to_file(profile_filename)
        
        print(f"\nRapport ydata-profiling généré et sauvegardé sous : '{profile_filename}'")
        print(f"Veuillez ouvrir le fichier '{profile_filename}' manuellement depuis votre explorateur de fichiers pour le consulter.")

    except ImportError:
        print("ERREUR d'importation: La librairie ydata-profiling n'est pas installée.")
        print("Veuillez l'installer (par exemple, avec : pip install ydata-profiling) dans l'environnement Python utilisé par VS Code.")
    except Exception as e:
        print(f"Une erreur générale est survenue lors de la génération du rapport ydata-profiling : {e}")
else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini. Veuillez exécuter les cellules précédentes pour charger 'df'.")

Génération du rapport avec ydata-profiling... Cela peut prendre quelques instants.


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:01<00:00, 10.34it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


Rapport ydata-profiling généré et sauvegardé sous : 'Pandas_Profiling_Report_Revenu_Marocains.html'
Veuillez ouvrir le fichier 'Pandas_Profiling_Report_Revenu_Marocains.html' manuellement depuis votre explorateur de fichiers pour le consulter.


## 1.6 Bilan de la Compréhension des Données

Après avoir examiné les statistiques descriptives (`df.info()`, `df.describe()`) et le rapport d'exploration détaillé généré par `ydata-profiling` (version v4.16.1), nous pouvons synthétiser nos principales observations concernant le dataset :

*   **Structure Générale :** Le dataset contient 40 000 instances et 18 attributs. Il a été confirmé qu'il n'y a **aucune ligne dupliquée**. Le nombre total de cellules manquantes est de 2871, ce qui représente 0.4% de l'ensemble des données.

*   **Variable Cible (`Revenu_Annuel`) :**
    *   Elle présente une distribution fortement asymétrique à droite : la moyenne est d'environ 23 235 DH (d'après `df.describe()` et le rapport), tandis que la médiane est significativement plus basse, à environ 9 420 DH (d'après `df.describe()`). Cette asymétrie est typique des données de revenu.
    *   Les valeurs s'étendent de 200 DH à 700 000 DH.
    *   Aucune valeur manquante n'est présente pour cette variable cible.

*   **Variables Explicatives Numériques :**
    *   `Age` : Les valeurs varient de -5 à 150 ans, confirmant la présence d'aberrations introduites (14 valeurs négatives). Le rapport signale une forte corrélation avec `Annees_experience` et `Categorie_age`.
    *   `Annees_experience` : Comporte 40 valeurs manquantes (0.1%). Un nombre notable de 5251 observations (13.1% de la colonne) présentent une valeur de zéro, indiquant des individus sans expérience professionnelle (potentiellement jeunes ou inactifs).

*   **Variables Explicatives Catégorielles et Textuelles :**
    *   `Categorie_age` : Comporte 40 valeurs manquantes (0.1%), probablement dues aux valeurs aberrantes de `Age` qui ne correspondent à aucun intervalle défini lors de sa création. Elle est logiquement très corrélée à `Age`.
    *   `Sexe`, `Milieu`, `Region_geographique`, `Niveau_education`, `CSP` : Ne présentent pas de valeurs manquantes. Leurs répartitions sont conformes à la logique de génération.
    *   `Etat_matrimonial` : Présente 35 valeurs manquantes (0.1%).
    *   `Secteur_emploi` : A le plus grand nombre de valeurs manquantes avec 2610 occurrences (6.5% des données pour cette colonne), ce qui est cohérent avec la logique de génération (les individus "Inactifs" n'ont pas de secteur d'emploi).
    *   `Propriete_immobiliere` (42 NaN, 0.1%), `Vehicule_motorise` (35 NaN, 0.1%), `Terrain_agricole` (31 NaN, 0.1%), `Revenu_secondaire` (38 NaN, 0.1%) : Toutes ces variables binaires présentent un faible pourcentage de valeurs manquantes.
    *   `Terrain_agricole` est signalé par `ydata-profiling` comme étant déséquilibré (la majorité des individus n'en possèdent pas, ce qui est confirmé par `df.describe()`).

*   **Colonnes Identifiants (Type Texte) :**
    *   `Adresse_Email` (39964 valeurs distinctes sur 40000) et `CIN` (39999 valeurs distinctes sur 40000) : Possèdent une très haute cardinalité (presque uniques pour chaque enregistrement).

*   **Colonnes à considérer pour suppression (Redondance/Non Pertinence) :**
    *   `Revenu_Mensuel` : Identifié comme parfaitement corrélé avec `Revenu_Annuel` (calculé comme `Revenu_Annuel / 12`). Sa suppression sera effectuée lors de la préparation des données pour éviter la redondance.
    *   `Adresse_Email` et `CIN` : En raison de leur haute cardinalité et de leur nature d'identifiant, elles sont considérées comme non pertinentes pour la prédiction du revenu. Leur suppression sera effectuée lors de la préparation des données.
    *   `Categorie_age` : Étant directement dérivée de `Age` et fortement corrélée, sa pertinence sera réévaluée. Si `Age` est conservé, `Categorie_age` pourrait être redondante pour certains modèles.

*   **Corrélations Notables (basées sur le rapport `ydata-profiling`) :**
    *   Les variables qui semblent les plus positivement corrélées avec `Revenu_Annuel` (basé sur la matrice de corrélation de Pearson fournie) incluent `CSP` (coefficient approx. 0.43), `Niveau_education` (0.34), `Propriete_immobiliere` (0.29), et `Vehicule_motorise` (0.26).
    *   Le rapport HTML complet de `ydata-profiling` devrait être consulté pour des métriques de corrélation plus adaptées aux variables mixtes (ex: Phik).
    *   Plusieurs corrélations significatives existent entre les variables explicatives, signalant une possible **multicollinéarité** :
        *   `Age` et `Annees_experience` (0.68)
        *   `CSP` et `Niveau_education` (0.64)
        *   `CSP` et `Terrain_agricole` (0.71, probablement car la CSP "Agriculteurs" est liée à la possession de terrain agricole)
        *   `CSP` est également corrélée avec `Propriete_immobiliere` (0.45), `Vehicule_motorise` (0.42), et `Secteur_emploi` (0.47).
        La gestion de cette multicollinéarité pourrait être nécessaire selon les modèles utilisés.

*   **Alertes du rapport `ydata-profiling` (Résumé) :** Les principales alertes concernent les fortes corrélations (entre prédicteurs et entre `Age` et `Categorie_age`), les valeurs manquantes (surtout dans `Secteur_emploi`), le déséquilibre de la variable `Terrain_agricole`, et le nombre important de zéros dans `Annees_experience`.

**Conclusion de l'EDA et Prochaines Étapes :**
L'exploration des données nous a fourni une compréhension approfondie du dataset, de ses caractéristiques individuelles, de la qualité des données (présence de NaN, d'outliers intentionnels, et de zéros) et des relations et corrélations potentielles entre les variables.
Les prochaines étapes se concentreront sur la préparation de ces données pour la modélisation :
1.  **Séparation des données** en ensembles d'apprentissage et de test.
2.  **Nettoyage des données**, ce qui inclura le traitement des valeurs manquantes et des valeurs aberrantes identifiées. (Les lignes dupliquées ne sont pas un problème ici).
3.  **Transformation des données**, comprenant la suppression des colonnes jugées non pertinentes ou redondantes, l'encodage approprié des variables catégorielles en format numérique, et potentiellement la normalisation ou la standardisation des variables numériques. La gestion de la multicollinéarité sera également considérée.
Ces opérations seront, autant que possible, intégrées dans des pipelines `scikit-learn` pour assurer un processus de prétraitement structuré, reproductible et applicable de manière cohérente aux différents sous-ensembles de données.

# 2. Préparation des Données

Après avoir exploré et compris notre dataset, cette section est dédiée à sa préparation en vue de la modélisation. Cela implique plusieurs étapes cruciales :
*   La suppression des colonnes jugées inutiles ou redondantes.
*   La séparation des données en ensembles d'apprentissage (`X_train`, `y_train`) et de test (`X_test`, `y_test`).
*   Le nettoyage des données, notamment le traitement des valeurs manquantes et des valeurs aberrantes.
*   La transformation des données, incluant l'encodage des variables catégorielles en format numérique et la normalisation/standardisation des variables numériques.

Conformément au cahier des charges, ces étapes de nettoyage et de transformation seront, autant que possible, intégrées dans des **pipelines `scikit-learn`** pour un traitement structuré et reproductible.

## 2.1 Suppression des Colonnes Inutiles et Définition des Features (X) et de la Cible (y)

Basé sur notre analyse exploratoire (section 1.6), nous allons d'abord supprimer les colonnes identifiées comme non pertinentes (identifiants uniques) ou redondantes. Ensuite, nous séparerons notre DataFrame en un ensemble de variables explicatives (features `X`) et la variable cible (`y`).

In [10]:
if 'df' in locals() and df is not None:
    # Copie du DataFrame original pour éviter de modifier l'original df si on veut y revenir
    df_processed = df.copy()

    # --- Définition de la variable cible ---
    TARGET_COL = 'Revenu_Annuel'
    y = df_processed[TARGET_COL]

    # --- Suppression des colonnes non pertinentes ou redondantes avant de définir X ---
    # Colonnes identifiées dans le bilan 1.6
    cols_to_drop_initial = ['Revenu_Mensuel', 'Adresse_Email', 'CIN']
    
    # On pourrait aussi décider ici de supprimer 'Categorie_age' si on juge qu'elle est
    # trop redondante avec 'Age' et qu'on ne veut pas la gérer spécifiquement.
    # Pour l'instant, laissons-la et nous verrons si notre pipeline de transformation la traite
    # ou si nous la retirons explicitement plus tard des listes de colonnes numériques/catégorielles.
    # Exemple si on voulait la supprimer ici aussi :
    # cols_to_drop_initial.append('Categorie_age')

    # S'assurer que les colonnes à supprimer existent bien dans le DataFrame
    actual_cols_to_drop = [col for col in cols_to_drop_initial if col in df_processed.columns]
    
    # Définition de X (variables explicatives)
    # On supprime la variable cible ET les colonnes identifiées pour suppression
    X = df_processed.drop(columns=[TARGET_COL] + actual_cols_to_drop, errors='ignore')

    print("Colonnes initialement supprimées de X :", actual_cols_to_drop)
    print(f"Nombre de colonnes restantes dans X avant séparation train/test : {X.shape[1]}")
    print("Liste des colonnes dans X :")
    print(X.columns.tolist())

    print(f"\nDimensions de X : {X.shape}")
    print(f"Dimensions de y : {y.shape}")

    # Afficher un aperçu de X et y
    print("\nAperçu de X (5 premières lignes) :")
    display(X.head())
    print("\nAperçu de y (5 premières valeurs) :")
    display(y.head())

else:
    print("ATTENTION : Le DataFrame 'df' n'est pas chargé ou n'est pas défini.")

Colonnes initialement supprimées de X : ['Revenu_Mensuel', 'Adresse_Email', 'CIN']
Nombre de colonnes restantes dans X avant séparation train/test : 14
Liste des colonnes dans X :
['Age', 'Categorie_age', 'Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'Annees_experience', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']

Dimensions de X : (40000, 14)
Dimensions de y : (40000,)

Aperçu de X (5 premières lignes) :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
0,56,Senior,Homme,Rural,Centre,Marié,Fondamental,5.0,Agriculteurs,Privé,Non,Non,Oui,Non
1,46,Senior,Femme,Urbain,Centre,Célibataire,Supérieur,14.0,Cadres supérieurs,Public,Oui,Oui,Non,Non
2,32,Adulte,Homme,Rural,Sud,Marié,Fondamental,8.0,Ouvriers,Informel,Non,Oui,Non,Non
3,60,Âgé,Homme,Rural,Sud,Marié,Supérieur,17.0,Cadres supérieurs,Privé,Oui,Oui,Non,Oui
4,25,Adulte,Femme,Urbain,Centre,Marié,Secondaire,3.0,Employés,Privé,Non,Non,Non,Non



Aperçu de y (5 premières valeurs) :


0      5231.0
1     86759.0
2      3941.0
3    109176.0
4     14906.0
Name: Revenu_Annuel, dtype: float64

In [11]:
from sklearn.model_selection import train_test_split

if 'X' in locals() and 'y' in locals(): # Vérifier que X et y existent
    # Utilisation de RANDOM_SEED pour la reproductibilité du split
    # (RANDOM_SEED a été défini au début du notebook, ex: RANDOM_SEED = 42)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.30, 
                                                        random_state=RANDOM_SEED) # Assurez-vous que RANDOM_SEED est défini

    print("Dimensions des ensembles de données après séparation :")
    print(f"  X_train: {X_train.shape}")
    print(f"  y_train: {y_train.shape}")
    print(f"  X_test: {X_test.shape}")
    print(f"  y_test: {y_test.shape}")

    print("\nProportion des données dans chaque ensemble :")
    print(f"  Apprentissage (train) : {len(X_train) / len(X):.0%}") # len(X) ou len(df_processed)
    print(f"  Test : {len(X_test) / len(X):.0%}")
    
    # Afficher un aperçu pour vérifier
    print("\nAperçu de X_train (5 premières lignes) :")
    display(X_train.head())

else:
    print("ATTENTION : Les variables X et/ou y ne sont pas définies. Veuillez exécuter la cellule précédente.")

Dimensions des ensembles de données après séparation :
  X_train: (28000, 14)
  y_train: (28000,)
  X_test: (12000, 14)
  y_test: (12000,)

Proportion des données dans chaque ensemble :
  Apprentissage (train) : 70%
  Test : 30%

Aperçu de X_train (5 premières lignes) :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire
38015,34,Adulte,Femme,Urbain,Sud,Célibataire,Secondaire,2.0,Employés,Privé,Oui,Non,Non,Non
2281,21,Jeune,Homme,Rural,Est,Célibataire,Fondamental,2.0,Agriculteurs,Informel,Oui,Oui,Oui,Non
36629,56,Senior,Homme,Rural,Ouest,Célibataire,Fondamental,19.0,Ouvriers,Informel,Non,Non,Non,Non
6087,25,Adulte,Femme,Urbain,Nord,Divorcé,Secondaire,2.0,Ouvriers,Privé,Non,Non,Non,Non
11792,35,Adulte,Homme,Urbain,Ouest,Marié,Secondaire,7.0,Employés,Privé,Non,Non,Non,Non


## 2.3 Identification des Types de Colonnes pour les Pipelines de Prétraitement

Avant de construire nos pipelines, nous devons identifier clairement quelles colonnes de notre ensemble d'apprentissage `X_train` sont numériques et lesquelles sont catégorielles. Ces deux types de colonnes nécessiteront des traitements différents (par exemple, imputation et normalisation pour les numériques, imputation et encodage pour les catégorielles).

Nous identifierons également la colonne `Age` séparément car elle contient des valeurs aberrantes spécifiques (-5, 150) que nous pourrions vouloir traiter d'une manière particulière. De même pour `Annees_experience` qui contient des zéros et des valeurs manquantes.

In [12]:
if 'X_train' in locals(): # Vérifier que X_train existe
    # Identifier les colonnes numériques
    # On peut exclure 'Age' et 'Annees_experience' pour un traitement spécifique si besoin,
    # mais pour l'instant, incluons-les dans la détection générale.
    numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()

    # Identifier les colonnes catégorielles
    # Note: 'Categorie_age' est de type 'object' d'après df.info(), donc elle sera ici.
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    print("Colonnes Numériques Identifiées :")
    print(numerical_cols)
    print(f"Nombre de colonnes numériques : {len(numerical_cols)}")

    print("\nColonnes Catégorielles Identifiées :")
    print(categorical_cols)
    print(f"Nombre de colonnes catégorielles : {len(categorical_cols)}")

    # Vérification : le total des colonnes numériques et catégorielles doit correspondre au nombre de colonnes dans X_train
    if len(numerical_cols) + len(categorical_cols) == X_train.shape[1]:
        print("\nLa somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.")
    else:
        print("\nATTENTION : Incohérence dans le nombre de colonnes. Vérifiez les types.")
        print(f"Total colonnes X_train: {X_train.shape[1]}")
        print(f"Numériques + Catégorielles: {len(numerical_cols) + len(categorical_cols)}")

    # Points d'attention spécifiques (basés sur l'EDA)
    # Colonne 'Age' (numérique) : contient des outliers (-5, 150)
    # Colonne 'Annees_experience' (numérique) : contient des NaN et beaucoup de zéros
    # Colonne 'Categorie_age' (catégorielle) : contient des NaN
    # Colonne 'Secteur_emploi' (catégorielle) : contient le plus de NaN

    # Vous pourriez vouloir affiner ces listes. Par exemple, si 'Categorie_age' était
    # une représentation ordinale d'Age, vous pourriez la traiter différemment.
    # Ici, elle est 'object' et sera traitée comme les autres catégorielles.

else:
    print("ATTENTION : X_train n'est pas défini. Veuillez exécuter les cellules précédentes.")

Colonnes Numériques Identifiées :
['Age', 'Annees_experience']
Nombre de colonnes numériques : 2

Colonnes Catégorielles Identifiées :
['Categorie_age', 'Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']
Nombre de colonnes catégorielles : 12

La somme des colonnes numériques et catégorielles correspond bien au nombre total de colonnes dans X_train.


## 2.4 Création des Pipelines de Prétraitement

Nous allons maintenant définir des pipelines distincts pour les variables numériques et catégorielles en utilisant `make_pipeline` et `ColumnTransformer` de Scikit-learn.

**Pour les variables numériques, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous utiliserons la médiane pour imputer les NaN, car elle est moins sensible aux outliers que la moyenne.
2.  **Traitement des valeurs aberrantes :** Pour la colonne `Age`, nous appliquerons une technique de "clipping" (plafonnement) pour ramener les valeurs aberrantes dans une plage plus raisonnable (par exemple, 18 à 70 ans). Pour les autres colonnes numériques, nous pourrions envisager `RobustScaler` qui est moins sensible aux outliers, ou d'autres techniques si nécessaire.
3.  **Normalisation/Standardisation (`StandardScaler` ou `RobustScaler`) :** Pour mettre toutes les variables numériques sur une échelle comparable, ce qui est important pour de nombreux algorithmes (Régression Linéaire, MLP, etc.). `StandardScaler` centre les données autour de 0 avec un écart-type de 1. `RobustScaler` utilise les quantiles et est plus robuste aux outliers.

**Pour les variables catégorielles, le pipeline inclura :**
1.  **Traitement des valeurs manquantes (`SimpleImputer`) :** Nous remplacerons les NaN par la valeur la plus fréquente de la colonne ou par une constante comme "Manquant".
2.  **Encodage (`OneHotEncoder`) :** Nous transformerons les catégories en un ensemble de colonnes binaires. Nous utiliserons `handle_unknown='ignore'` pour gérer les catégories rares qui pourraient apparaître dans l'ensemble de test mais pas dans l'ensemble d'apprentissage.

Ces pipelines individuels seront ensuite combinés à l'aide d'un `ColumnTransformer`.

In [13]:
from sklearn.pipeline import Pipeline, make_pipeline # make_pipeline n'est pas utilisé ici mais bon import
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin # Pour les transformateurs personnalisés
import pandas as pd # S'assurer que pandas est importé si AgeClipper l'utilise
import numpy as np # S'assurer que numpy est importé

# Vérifier que les variables nécessaires existent (issues des cellules précédentes)
if ('X_train' in locals() and 
    'numerical_cols' in locals() and 
    'categorical_cols' in locals()):

    # --- Transformateur personnalisé pour le clipping de l'âge ---
    class AgeClipper(BaseEstimator, TransformerMixin):
        def __init__(self, min_age=18, max_age=75): # Valeurs ajustées
            self.min_age = min_age
            self.max_age = max_age
        def fit(self, X, y=None):
            # X est un DataFrame ici car ColumnTransformer passe les colonnes sélectionnées comme DataFrame
            return self
        def transform(self, X, y=None):
            X_transformed = X.copy()
            # X est un DataFrame avec une ou plusieurs colonnes. On clip chaque colonne passée.
            for col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].clip(lower=self.min_age, upper=self.max_age)
            return X_transformed

    # --- Définition des pipelines pour chaque type de colonne ---
    
    # Les listes de colonnes ont été définies dans la cellule précédente :
    # numerical_cols = ['Age', 'Annees_experience']
    # categorical_cols = ['Categorie_age', 'Sexe', ..., 'Revenu_secondaire']

    # Pipeline pour la colonne 'Age'
    age_pipeline = Pipeline([
        ('imputer_age', SimpleImputer(strategy='median')), 
        ('clipper', AgeClipper(min_age=18, max_age=75)), 
        ('scaler_age', StandardScaler()) 
    ])

    # Pipeline pour la colonne 'Annees_experience'
    experience_pipeline = Pipeline([
        ('imputer_exp', SimpleImputer(strategy='median')),
        ('scaler_exp', RobustScaler()) 
    ])

    # Pipeline pour les variables catégorielles
    categorical_pipeline = Pipeline([
        ('imputer_cat', SimpleImputer(strategy='most_frequent')), 
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop=None)) # drop=None par défaut, mais explicite
    ])

    # --- Combinaison des pipelines avec ColumnTransformer ---
    
    # Listes explicites pour ColumnTransformer
    age_col_list = ['Age'] # La colonne à laquelle appliquer age_pipeline
    experience_col_list = ['Annees_experience'] # La colonne pour experience_pipeline
    # categorical_cols est déjà la liste des colonnes catégorielles

    # S'assurer que les colonnes existent bien dans X_train avant de les utiliser
    # (Normalement, c'est le cas si l'identification a bien fonctionné)
    final_age_col = [col for col in age_col_list if col in X_train.columns]
    final_experience_col = [col for col in experience_col_list if col in X_train.columns]
    final_categorical_cols = [col for col in categorical_cols if col in X_train.columns]

    # Construire la liste des transformateurs
    transformers_for_columntransformer = []
    if final_age_col:
        transformers_for_columntransformer.append(('age_processing', age_pipeline, final_age_col))
    if final_experience_col:
        transformers_for_columntransformer.append(('experience_processing', experience_pipeline, final_experience_col))
    if final_categorical_cols:
        transformers_for_columntransformer.append(('categorical_processing', categorical_pipeline, final_categorical_cols))
    
    if transformers_for_columntransformer:
        preprocessor = ColumnTransformer(
            transformers=transformers_for_columntransformer,
            remainder='drop' # Les colonnes non listées explicitement ici seront supprimées.
                             # Comme nous avons listé toutes les colonnes de X_train (via numerical_cols et categorical_cols),
                             # remainder='drop' ou 'passthrough' (s'il n'y a pas d'autres colonnes) donnerait le même résultat
                             # en termes de colonnes conservées pour la transformation.
        )
        
        print("Preprocessor ColumnTransformer créé avec succès.")

        # Afficher le preprocessor pour vérifier sa structure
        from sklearn import set_config
        set_config(display='diagram') 
        display(preprocessor)
        set_config(display='text') # Revenir à l'affichage texte par défaut
    else:
        print("ATTENTION : Aucun transformateur n'a été ajouté au ColumnTransformer. Vérifiez les listes de colonnes.")
        preprocessor = None # S'assurer que preprocessor est None si rien n'est fait

else:
    print("ATTENTION : X_train ou les listes de colonnes numerical_cols/categorical_cols ne sont pas définis. Veuillez exécuter les cellules précédentes.")

Preprocessor ColumnTransformer créé avec succès.


ColumnTransformer(transformers=[('age_processing',
                                 Pipeline(steps=[('imputer_age',
                                                  SimpleImputer(strategy='median')),
                                                 ('clipper', AgeClipper()),
                                                 ('scaler_age',
                                                  StandardScaler())]),
                                 ['Age']),
                                ('experience_processing',
                                 Pipeline(steps=[('imputer_exp',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler_exp',
                                                  RobustScaler())]),
                                 ['Annees_experience']),
                                ('categorical_processing',
                                 Pipeline(steps=[('imputer_cat',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Categorie_age', 'Sexe', 'Milieu',
                                  'Region_geographique', 'Etat_matrimonial',
                                  'Niveau_education', 'CSP', 'Secteur_emploi',
                                  'Propriete_immobiliere', 'Vehicule_motorise',
                                  'Terrain_agricole', 'Revenu_secondaire'])])

## 2.5 Application du Preprocessor et Récupération des Noms de Features

Le `preprocessor` étant défini, nous allons maintenant l'ajuster (`fit`) sur l'ensemble d'entraînement `X_train` et l'utiliser pour transformer (`transform`) à la fois `X_train` et `X_test`.

Il est important de récupérer les noms des nouvelles features générées, notamment par le `OneHotEncoder`, pour pouvoir créer des DataFrames à partir des arrays NumPy retournés et inspecter les données transformées.

In [16]:
# Cellule de CODE à placer sous le Markdown "2.5 Application du Preprocessor..."

if ('preprocessor' in locals() and preprocessor is not None and 
    'X_train' in locals() and 'X_test' in locals() and
    'categorical_cols' in locals()): # Assurer que categorical_cols est défini

    print("Application du preprocessor sur X_train (fit_transform)...")
    # Ajuster et transformer X_train
    X_train_processed_np = preprocessor.fit_transform(X_train)
    
    print("Application du preprocessor sur X_test (transform)...")
    # Transformer X_test avec le preprocessor déjà ajusté sur X_train
    X_test_processed_np = preprocessor.transform(X_test)

    # --- Récupération des noms de features ---
    numeric_feature_names_processed = []
    # L'ordre doit correspondre à celui dans ColumnTransformer.
    # Le premier transformateur numérique est 'age_processing'
    if 'age_processing' in preprocessor.named_transformers_:
         numeric_feature_names_processed.append('Age_Processed') 
    # Le deuxième transformateur numérique est 'experience_processing'
    if 'experience_processing' in preprocessor.named_transformers_:
         numeric_feature_names_processed.append('Annees_Experience_Processed')
        
    try:
        # Récupérer les noms des colonnes catégorielles après OneHotEncoding
        # 'categorical_processing' est le nom du transformateur pour les catégorielles dans ColumnTransformer
        # 'onehot' est le nom de l'étape OneHotEncoder dans le pipeline 'categorical_pipeline'
        onehot_transformer = preprocessor.named_transformers_['categorical_processing']
        onehot_encoder_step = onehot_transformer.named_steps['onehot']
        # 'categorical_cols' est la liste des noms de colonnes catégorielles originales passées à ce pipeline
        onehot_categorical_feature_names = list(onehot_encoder_step.get_feature_names_out(categorical_cols))
    except KeyError as e:
        print(f"Erreur de clé lors de la récupération des noms de features du OneHotEncoder: {e}")
        print("Vérifiez les noms des étapes ('categorical_processing', 'onehot') dans votre ColumnTransformer et le pipeline associé.")
        onehot_categorical_feature_names = [] 
    except Exception as e:
        print(f"Erreur générale lors de la récupération des noms de features du OneHotEncoder : {e}")
        onehot_categorical_feature_names = []

    # Combiner tous les noms de features dans l'ordre de sortie du ColumnTransformer
    final_feature_names = numeric_feature_names_processed + onehot_categorical_feature_names

    # Créer des DataFrames à partir des arrays NumPy transformés
    X_train_processed_df = pd.DataFrame(X_train_processed_np, 
                                        columns=final_feature_names, 
                                        index=X_train.index) # Conserver les index originaux
    
    X_test_processed_df = pd.DataFrame(X_test_processed_np, 
                                       columns=final_feature_names, 
                                       index=X_test.index) # Conserver les index originaux

    print(f"\nDimensions de X_train_processed_df : {X_train_processed_df.shape}")
    print("Aperçu de X_train_processed_df (5 premières lignes) :")
    display(X_train_processed_df.head())

    print(f"\nDimensions de X_test_processed_df : {X_test_processed_df.shape}")
    # print("Aperçu de X_test_processed_df (5 premières lignes) :") # Optionnel pour l'instant
    # display(X_test_processed_df.head())

    print(f"\nNombre de NaN dans X_train_processed_df : {X_train_processed_df.isnull().sum().sum()}")
    print(f"Nombre de NaN dans X_test_processed_df : {X_test_processed_df.isnull().sum().sum()}")

else:
    print("ATTENTION : Des variables nécessaires (preprocessor, X_train, X_test, categorical_cols) ne sont pas définies. Veuillez exécuter les cellules précédentes.")

Application du preprocessor sur X_train (fit_transform)...
Application du preprocessor sur X_test (transform)...

Dimensions de X_train_processed_df : (28000, 40)
Aperçu de X_train_processed_df (5 premières lignes) :


,Age_Processed,Annees_Experience_Processed,Categorie_age_Adulte,Categorie_age_Jeune,Categorie_age_Senior,Categorie_age_Âgé,Sexe_Femme,Sexe_Homme,Milieu_Rural,Milieu_Urbain,Region_geographique_Centre,Region_geographique_Est,Region_geographique_Nord,Region_geographique_Ouest,Region_geographique_Sud,Etat_matrimonial_Célibataire,Etat_matrimonial_Divorcé,Etat_matrimonial_Marié,Etat_matrimonial_Veuf,Niveau_education_Fondamental,Niveau_education_Sans niveau,Niveau_education_Secondaire,Niveau_education_Supérieur,CSP_Agriculteurs,CSP_Cadres supérieurs,CSP_Employés,CSP_Inactifs,CSP_Ouvriers,CSP_Professions intermédiaires,Secteur_emploi_Informel,Secteur_emploi_Privé,Secteur_emploi_Public,Propriete_immobiliere_Non,Propriete_immobiliere_Oui,Vehicule_motorise_Non,Vehicule_motorise_Oui,Terrain_agricole_Non,Terrain_agricole_Oui,Revenu_secondaire_Non,Revenu_secondaire_Oui
38015,-0.488069,-0.357143,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2281,-1.472609,-0.357143,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
36629,1.178075,0.857143,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6087,-1.169674,-0.357143,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
11792,-0.412336,0.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0



Dimensions de X_test_processed_df : (12000, 40)

Nombre de NaN dans X_train_processed_df : 0
Nombre de NaN dans X_test_processed_df : 0


# 3. Création et Validation des Modèles

Après avoir préparé nos données, nous passons maintenant à l'étape de modélisation. L'objectif est de :
1.  Sélectionner plusieurs algorithmes de régression.
2.  Ajuster (fine-tune) leurs hyperparamètres en utilisant la technique de validation croisée sur l'ensemble d'entraînement (`X_train_processed_df`, `y_train`).
3.  Comparer les performances des différents modèles sur la base des métriques MAE, RMSE, et R².
4.  Choisir le modèle le plus performant.

Les modèles à considérer sont : Régression Linéaire, Arbres de Décision, Forêts d'Arbres Décisionnels, Gradient Boosting, et Réseaux de Neurones Multi-couches (MLPRegressor).

## 3.1 Importation des Modèles, des Métriques et Définition des Grilles d'Hyperparamètres

Nous commençons par importer les classes des modèles de régression de Scikit-learn, les fonctions pour les métriques d'évaluation (MAE, RMSE, R²), et l'outil pour la recherche d'hyperparamètres (`GridSearchCV` ou `RandomizedSearchCV`).

Ensuite, nous allons définir un dictionnaire contenant les modèles que nous souhaitons tester ainsi que les grilles d'hyperparamètres à explorer pour chacun, conformément aux spécifications du projet.

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV # Ou RandomizedSearchCV si le temps est un facteur
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import numpy as np # Pour np.sqrt pour RMSE
import pandas as pd # Pour afficher les résultats
import time # Pour mesurer le temps d'entraînement

# S'assurer que nos données prétraitées sont disponibles
if ('X_train_processed_df' not in locals() or 
    'y_train' not in locals() or 
    'X_test_processed_df' not in locals() or 
    'y_test' not in locals()):
    print("ATTENTION: Les données prétraitées (X_train_processed_df, y_train, etc.) ne sont pas définies.")
    print("Veuillez exécuter les cellules de prétraitement des données avant de continuer.")
else:
    print("Données prétraitées prêtes pour la modélisation.")
    # Convertir les y en array NumPy si ce n'est pas déjà le cas (GridSearchCV peut parfois le préférer)
    # y_train est une Series Pandas, ce qui est généralement bien accepté.
    # y_train_values = y_train.values
    # y_test_values = y_test.values

# --- Définition des modèles et de leurs grilles d'hyperparamètres ---
# Remarque : Le préfixe pour les hyperparamètres dans un pipeline est 'nomdelétape__nomduparamètre'
# Puisque nous allons entraîner les modèles directement sur X_train_processed_df (sans pipeline supplémentaire ici,
# car le preprocessing est déjà fait), nous n'avons pas besoin de ce préfixe pour l'instant.
# Si nous devions mettre le modèle LUI-MÊME dans un nouveau pipeline (par ex. pour combiner avec d'autres étapes),
# alors le préfixe serait nécessaire. Pour l'instant, c'est plus simple.

models_and_params = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {} # Aucun hyperparamètre spécifié pour LinearRegression
    },
    'DecisionTreeRegressor': {
        'model': DecisionTreeRegressor(random_state=RANDOM_SEED),
        'params': {
            'criterion': ['squared_error'],
            'max_depth': [None, 5, 7, 10], # Réduit de 6 pour aller plus vite, vous pouvez garder celle du sujet
            'min_samples_split': [2, 5, 10] # Réduit de 3, 4
        }
    },
    'RandomForestRegressor': {
        'model': RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1), # n_jobs=-1 pour utiliser tous les coeurs
        'params': {
            'n_estimators': [50, 100, 150], # Réduit de 200
            'criterion': ['squared_error'],
            'max_depth': [None, 10, 15] # Réduit de 5, 20
        }
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(random_state=RANDOM_SEED),
        'params': {
            'loss': ['squared_error'], # 'huber', 'quantile' aussi possibles
            'learning_rate': [0.01, 0.1], # Réduit de 0.2
            'n_estimators': [100, 200], # Réduit de 300
            'subsample': [0.8, 1.0] # Réduit de 0.5
        }
    },
    'MLPRegressor': {
        'model': MLPRegressor(random_state=RANDOM_SEED, max_iter=300, early_stopping=True, n_iter_no_change=10), # max_iter augmenté, early_stopping ajouté
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (100, 50)], # Réduit de (100,100)
            'activation': ['relu', 'tanh'], # Réduit de 'logistic'
            'solver': ['adam'], # 'sgd' peut être très lent sans bon tuning du learning rate
            'alpha': [0.0001, 0.001], # Réduit de 0.01
            'learning_rate': ['constant', 'adaptive'] # 'invscaling' est souvent pour SGD
            # 'learning_rate_init': [0.001, 0.01], # learning_rate_init est souvent ajusté avec solver='sgd'
            # 'max_iter': [100, 200, 300] # max_iter est déjà dans le modèle, on peut le mettre ici aussi pour la grille
        }
    }
}

print("\nModèles et grilles d'hyperparamètres définis.")
# Afficher les clés pour vérifier
print("Modèles à tester :", list(models_and_params.keys()))

Données prétraitées prêtes pour la modélisation.

Modèles et grilles d'hyperparamètres définis.
Modèles à tester : ['LinearRegression', 'DecisionTreeRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'MLPRegressor']


In [ ]:
# Dictionnaire pour stocker les résultats
grid_search_results = {}

# Utiliser X_train_processed_df qui est un DataFrame Pandas.
# GridSearchCV s'attend généralement à des arrays NumPy ou des DataFrames Pandas.
# y_train est une Series Pandas, ce qui est bien.

# Définir le nombre de folds pour la validation croisée
CV_FOLDS = 5 # Vous pouvez réduire à 3 pour aller plus vite pendant les tests initiaux

if 'models_and_params' in locals() and 'X_train_processed_df' in locals() and 'y_train' in locals():
    for model_name, mp in models_and_params.items():
        print(f"--- Entraînement et ajustement pour : {model_name} ---")
        start_time = time.time()
        
        # Initialiser GridSearchCV
        # Nous utilisons 'r2' comme score principal pour la recherche.
        
        # Si vous voulez utiliser RandomizedSearchCV pour aller plus vite sur de grandes grilles :
        # from sklearn.model_selection import RandomizedSearchCV
        # search_cv = RandomizedSearchCV(mp['model'], mp['params'], cv=CV_FOLDS, scoring='r2', n_iter=10, random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
        # n_iter contrôle le nombre de combinaisons testées par RandomizedSearchCV

        search_cv = GridSearchCV(mp['model'], mp['params'], 
                                 cv=CV_FOLDS, 
                                 scoring='r2', 
                                 n_jobs=-1, # Utiliser tous les coeurs si possible
                                 verbose=1) # verbose=1 pour avoir des messages pendant l'entraînement
        
        try:
            # Entraîner sur les données prétraitées
            search_cv.fit(X_train_processed_df, y_train) 
            
            end_time = time.time()
            training_time = end_time - start_time
            
            # Stocker les résultats importants
            grid_search_results[model_name] = {
                'best_estimator': search_cv.best_estimator_,
                'best_params': search_cv.best_params_,
                'best_score_r2_cv': search_cv.best_score_,
                'training_time_seconds': training_time,
                'cv_results_all': search_cv.cv_results_ # Stocker tous les résultats de la CV
            }
            
            print(f"Meilleur score R² (Validation Croisée) pour {model_name} : {search_cv.best_score_:.4f}")
            print(f"Meilleurs hyperparamètres : {search_cv.best_params_}")
            print(f"Temps d'entraînement : {training_time:.2f} secondes")
            
        except Exception as e:
            print(f"ERREUR lors de l'entraînement de {model_name} : {e}")
            grid_search_results[model_name] = {
                'best_estimator': None,
                'best_params': None,
                'best_score_r2_cv': -np.inf, 
                'training_time_seconds': time.time() - start_time,
                'error_message': str(e),
                'cv_results_all': None
            }
        print("-" * 50)

    print("\n=== Fin de l'ajustement des hyperparamètres pour tous les modèles ===")
else:
    print("ATTENTION: Des variables nécessaires (models_and_params, X_train_processed_df, y_train) ne sont pas définies.")

--- Entraînement et ajustement pour : LinearRegression ---
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Meilleur score R² (Validation Croisée) pour LinearRegression : 0.8409
Meilleurs hyperparamètres : {}
Temps d'entraînement : 5.43 secondes
--------------------------------------------------
--- Entraînement et ajustement pour : DecisionTreeRegressor ---
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Meilleur score R² (Validation Croisée) pour DecisionTreeRegressor : 0.8904
Meilleurs hyperparamètres : {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_split': 2}
Temps d'entraînement : 5.97 secondes
--------------------------------------------------
--- Entraînement et ajustement pour : RandomForestRegressor ---
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Meilleur score R² (Validation Croisée) pour RandomForestRegressor : 0.8767
Meilleurs hyperparamètres : {'criterion': 'squared_error', 'max_depth': 10, 'n_estimators': 150}
Temps d'en